<a href="https://colab.research.google.com/github/Jdean4747/Data5600HW/blob/main/FinalProjectData5600.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from pycaret.classification import *

train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')
passenger_ids = test['PassengerId']

train['source'] = 'train'
test['source'] = 'test'
test['Survived'] = -1
data = pd.concat([train, test])

data['Title'] = data['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
data['Title'] = data['Title'].replace(['Lady','Countess','Capt','Col','Don','Dr','Major','Rev','Sir','Jonkheer','Dona'], 'Rare')
data['Title'] = data['Title'].replace(['Mlle', 'Ms'], 'Miss')
data['Title'] = data['Title'].replace('Mme', 'Mrs')
data['FamilySize'] = data['SibSp'] + data['Parch'] + 1
data['IsAlone'] = (data['FamilySize'] == 1).astype(int)
data['Age'] = data.groupby('Title')['Age'].transform(lambda x: x.fillna(x.median()))
data['Fare'] = data.groupby('Pclass')['Fare'].transform(lambda x: x.fillna(x.median()))
data['Embarked'].fillna(data['Embarked'].mode()[0], inplace=True)

data.drop(['Cabin', 'Ticket', 'Name'], axis=1, inplace=True)
train = data[data['source'] == 'train'].drop(['source'], axis=1)
test = data[data['source'] == 'test'].drop(['source', 'Survived'], axis=1)

clf = setup(data=train, target='Survived', session_id=42, use_gpu=False)
top3 = compare_models(n_select=3)
tuned_models = [tune_model(model) for model in top3]
blended = blend_models(estimator_list=tuned_models)
stacked = stack_models(estimator_list=[blended] + tuned_models)
final_model = finalize_model(stacked)
preds = predict_model(final_model, data=test)

submission = pd.DataFrame({'PassengerId': passenger_ids,'Survived': preds['prediction_label'].astype(int)})

,Description,Value
0,Session id,42
1,Target,Survived
2,Target type,Binary
3,Original data shape,"(891, 12)"
4,Transformed data shape,"(891, 18)"
5,Transformed train set shape,"(623, 18)"
6,Transformed test set shape,"(268, 18)"
7,Numeric features,8
8,Categorical features,3
9,Preprocess,True


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.8298,0.8622,0.7074,0.8277,0.7600,0.6298,0.6370,0.2090
lr,Logistic Regression,0.8218,0.8594,0.7368,0.7909,0.7595,0.6188,0.6228,0.9800
ridge,Ridge Classifier,0.8170,0.8603,0.7284,0.7832,0.7527,0.6081,0.6112,0.0800
lda,Linear Discriminant Analysis,0.8170,0.8605,0.7243,0.7855,0.7517,0.6076,0.6106,0.0800
rf,Random Forest Classifier,0.8105,0.8467,0.7116,0.7821,0.7412,0.5929,0.5982,0.3280
ada,Ada Boost Classifier,0.8090,0.8359,0.7080,0.7822,0.7357,0.5881,0.5965,0.1730
xgboost,Extreme Gradient Boosting,0.8059,0.8358,0.6909,0.7856,0.7292,0.5800,0.5880,0.1650
lightgbm,Light Gradient Boosting Machine,0.8011,0.8352,0.6826,0.7756,0.7225,0.5692,0.5750,0.4000
nb,Naive Bayes,0.7978,0.8338,0.7618,0.7285,0.7428,0.5767,0.5792,0.0820
et,Extra Trees Classifier,0.7913,0.8254,0.6822,0.7586,0.7152,0.5515,0.5564,0.2360


Processing:   0%|          | 0/67 [00:00<?, ?it/s]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8571,0.8616,0.7500,0.8571,0.8000,0.6897,0.6934
1,0.8254,0.8665,0.7083,0.8095,0.7556,0.6207,0.6240
2,0.8571,0.8905,0.7083,0.8947,0.7907,0.6845,0.6952
3,0.8387,0.9169,0.8261,0.7600,0.7917,0.6605,0.6620
4,0.8226,0.8191,0.5833,0.9333,0.7179,0.5984,0.6335
5,0.8871,0.9200,0.7917,0.9048,0.8444,0.7565,0.7606
6,0.7742,0.8547,0.5833,0.7778,0.6667,0.5011,0.5130
7,0.8065,0.8010,0.5833,0.8750,0.7000,0.5654,0.5908
8,0.8548,0.8339,0.7083,0.8947,0.7907,0.6819,0.6928


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

...

Fitting 10 folds for each of 10 candidates, totalling 100 fits


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8095,0.0000,0.7917,0.7308,0.7600,0.6025,0.6039
1,0.8254,0.0000,0.7500,0.7826,0.7660,0.6268,0.6272
2,0.8254,0.0000,0.7500,0.7826,0.7660,0.6268,0.6272
3,0.8387,0.0000,0.8261,0.7600,0.7917,0.6605,0.6620
4,0.8387,0.0000,0.6250,0.9375,0.7500,0.6379,0.6664
5,0.8710,0.0000,0.7917,0.8636,0.8261,0.7238,0.7256
6,0.7742,0.0000,0.6250,0.7500,0.6818,0.5090,0.5141
7,0.8387,0.0000,0.7083,0.8500,0.7727,0.6493,0.6558
8,0.8065,0.0000,0.7083,0.7727,0.7391,0.5857,0.5871


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8095,0.8835,0.7917,0.7308,0.7600,0.6025,0.6039
1,0.8254,0.8750,0.7500,0.7826,0.7660,0.6268,0.6272
2,0.8254,0.8932,0.7500,0.7826,0.7660,0.6268,0.6272
3,0.8387,0.8986,0.8261,0.7600,0.7917,0.6605,0.6620
4,0.8387,0.8147,0.6250,0.9375,0.7500,0.6379,0.6664
5,0.8710,0.9287,0.7917,0.8636,0.8261,0.7238,0.7256
6,0.7742,0.8607,0.6250,0.7500,0.6818,0.5090,0.5141
7,0.8226,0.8300,0.6667,0.8421,0.7442,0.6112,0.6209
8,0.8065,0.8257,0.7083,0.7727,0.7391,0.5857,0.5871


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
submission.to_csv('titanic6.csv', index=False)
from google.colab import files
files.download('titanic6.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>